In [1]:
import plotly.express as px
import requests
import pandas as pd
import numpy as np

In [2]:
stats = pd.read_excel("SleeperData.xlsx")
stats

,Owner,Week,Player,player_id,age,position,gp,off_snp,tm_off_snp,pts_ppr,...,pass_td,sacks,fum_rec,int,def_td,yds_allow,pts_allow,blk_kick,fum,fum_lost
0,keaton22,1,Cole Beasley,1144,32.0,WR,1.0,77.0,85.0,14.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,keaton22,1,Keenan Allen,1479,29.0,WR,1.0,67.0,81.0,18.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
2,keaton22,1,Devonta Freeman,2168,29.0,RB,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,keaton22,1,Melvin Gordon,2320,28.0,RB,1.0,33.0,66.0,20.80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,keaton22,1,Dak Prescott,3294,28.0,QB,1.0,83.0,83.0,27.42,...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3275,KingDarsh,Season,Brandon Aiyuk,6803,23.0,WR,17.0,841.0,1021.0,168.30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0
3276,KingDarsh,Season,James Robinson,6955,23.0,RB,14.0,530.0,909.0,169.90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,2.0
3277,KingDarsh,Season,Najee Harris,7528,23.0,RB,17.0,940.0,1097.0,300.70,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3278,KingDarsh,Season,Buffalo Bills,BUF,NaN,DEF,17.0,NaN,NaN,157.00,...,NaN,NaN,8.0,19.0,1.0,4637.0,287.0,2.0,NaN,NaN


# Avg Age by Position

In [38]:
age2 = stats[stats["Week"] == "Season"][["Owner", "age", "position", "Week"]]
age2 = age2.groupby(["Owner", "position"]).agg({"age":np.mean})
age2 = age2.reset_index()
fig = px.bar(age2, y = "age", x = "Owner", color = "position", barmode = "group")
fig.update_layout(xaxis={'categoryorder':'total ascending'})
fig.update_yaxes(range = [18,45])
fig.update_layout(title_text="Average Age by Position", title_x=0.5)
fig.update_layout(
    font_family="Times New Roman",
    font_color="black",
    title_font_family="Times New Roman",
    title_font_color="blue",
    title_font_size=24
)
fig.show()

# Avg Team Age

In [32]:
avg_age = stats[stats["Week"] == "Season"].age.mean()
age = pd.DataFrame(stats[stats["Week"] == "Season"].groupby("Owner").agg({"age": np.mean}))

fig = px.bar(age, x = "age", color = 'age') #can drop color option, but I like how it ranks everyone's age and shows a clear legend

fig.add_vline(x=avg_age)
fig.update_layout(yaxis={'categoryorder':'total ascending'})
fig.update_xaxes(range = [18,30])
fig.update_layout(title_text="Average Age by Owner's Teams", title_x=0.5)
fig.update_layout(
    font_family="Times New Roman",
    font_color="black",
    title_font_family="Times New Roman",
    title_font_color="blue",
    title_font_size=24
)
fig.show()

# Individual Player's Trends Throughout the Year
# found from players they own at end of the season

In [37]:
username = "ikei" #use variable entered by user on site
users_stats = stats[stats["Owner"] == username]
users_wkly_stats = users_stats[users_stats["Week"] != "Season"]

fig = px.line(users_wkly_stats, x = "Week", y = "pts_half_ppr", facet_col = "Player", facet_col_wrap=4,
            width=1000, height=800)
fig.update_layout(title_text=f"{username}'s Player Trends Throughout the Year", title_x=0.5)
fig.update_layout(
    font_family="Times New Roman",
    font_color="black",
    title_font_family="Times New Roman",
    title_font_color="blue",
    title_font_size=24
)
fig.show()

# avg ypc,  tds, ypr, ppg by user's rb

In [62]:
username = "ikei" #use variable entered by user on site
point_settings = 'pts_half_ppr' #have user select this during league selection or pull in the scoring stats for their league

#create dataframes that can filter by owner, then season, and then by position
users_stats = stats[stats["Owner"] == username]
user_szn_stats = users_stats[stats['Week'] == 'Season']
rb_stats = user_szn_stats[stats["position"] == "RB"]

#calculate desired stats
rb_stats['ypc'] = rb_stats['rush_yd'] / rb_stats['rush_att']

rb_stats['ypr'] = rb_stats['rec_yd'] / rb_stats['rec']

rb_stats['rush_tds_per_game'] = rb_stats['rush_td'] / rb_stats['gp']
#if they never had a rush td set it to 0
rb_stats['rush_tds_per_game'] = np.where(rb_stats['rush_tds_per_game'].isnull(), 0, rb_stats['rush_tds_per_game'])

rb_stats['rec_tds_per_game'] = rb_stats['rec_td'] / rb_stats['gp']
#if they never had a rec td set it to 0
rb_stats['rec_tds_per_game'] = np.where(rb_stats['rec_tds_per_game'].isnull(), 0, rb_stats['rec_tds_per_game'])

rb_stats['ppg'] = rb_stats[point_settings] / rb_stats['gp']

#separate stats into two dif dfs for heatmap visualization
rb_stats1 = rb_stats[['Player', 'ypc', 'ypr']]
rb_stats1 = rb_stats1.set_index('Player')
rb_stats2 = rb_stats[['Player', 'rush_tds_per_game', 'rec_tds_per_game']]
rb_stats2 = rb_stats2.set_index('Player')

<ipython-input-62-b776567108fe>:6: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

<ipython-input-62-b776567108fe>:7: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

<ipython-input-62-b776567108fe>:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-62-b776567108fe>:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-62-b776567108fe>:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a Data

In [60]:
fig = px.imshow(rb_stats1, text_auto = True)
fig.show()

In [63]:
fig = px.imshow(rb_stats2, text_auto = True)
fig.show()

In [67]:
avg_ppg = rb_stats.ppg.mean()
fig = px.bar(rb_stats, x = 'Player', y = 'ppg', color = 'ppg')
fig.add_hline(y=avg_ppg)
fig.show()

In [73]:
rb_stats['tot_ypg'] = (rb_stats['rec_yd'] + rb_stats['rush_yd']) / rb_stats['gp']
tot_ypg = rb_stats.tot_ypg.mean()

fig = px.bar(rb_stats,x = 'Player', y = 'tot_ypg', color = 'tot_ypg')
fig.add_hline(y=tot_ypg)
fig.show()

<ipython-input-73-e23ff88f692b>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

